# Heat transfer in a fin with piecewise variation of some parameters


## Setup


In [ ]:
import warnings

import dill
import numpy as np
from boilercore.notebooks import disp, disp_free
from sympy import Eq, FiniteSet, Piecewise, Subs, dsolve, lambdify, pi
from uncertainties import umath

from boilerdata.models.params import PARAMS

from boilerdata.syms import (  # isort: skip
    params,
    x,
    T_s,
    q_s,
    h_a,
    k,
    h_w,
    inputs,
    r,
    T_infa,
    T_infw,
    x_s,
    x_wa,
    intermediate_vars,
    h,
    q_0,
    q_wa,
    T_0,
    T_inf,
    T_wa,
    x_0,
    functions,
    T_int,
    T_a,
    T_w,
    T,
)

## Parameters

Paramaters for this analysis.


In [ ]:
for key, val in {
    "Model parameters": params,
    "Model inputs": inputs,
    "Intermediate variables": intermediate_vars,
}.items():
    disp(key, FiniteSet(*val))

disp("Functions", FiniteSet(*(fun(x) for fun in functions)))

## General ODE and its solution


In [ ]:
P = 2 * pi * r
A_c = pi * r**2

ode = Eq(
    T(x).diff(x, 2) - h * P / k / A_c * (T(x) - T_inf),
    0,
)
ics = {
    T(x_0): T_0,
    Subs(T(x).diff(x), x, x_0): q_0 / k,
}
disp("ODE", ode)
disp("Initial conditions", *(Eq(lhs, rhs) for lhs, rhs in ics.items()))

In [ ]:
T_int_expr = dsolve(
    ode,
    T(x),
    ics=ics,
).rhs  # type: ignore  # sympy
disp_free("General solution to the ODE", Eq(T_int(x), T_int_expr))

## Solution in the water domain


In [ ]:
T_w_expr = T_int_expr.subs(
    {
        h: h_w,
        q_0: q_s,
        T_0: T_s,
        T_inf: T_infw,
        x_0: x_s,
    }
)

disp_free("Solution in the water domain", Eq(T_w(x), T_w_expr))

## Values at the domain boundary


In [ ]:
T_wa_expr_w = T_w_expr.subs(x, x_wa)
q_wa_expr_w = (
    T_w_expr.diff(x).subs(  # type: ignore  # sympy
        x,
        x_wa,
    )
    * k
)

disp_free("Temperature at the domain transition", Eq(T_wa, T_wa_expr_w))
disp_free("Heat flux at the domain transition", Eq(q_wa, q_wa_expr_w))

## Solution in the air domain


In [ ]:
T_a_int_expr = T_int_expr.subs(
    {
        h: h_a,
        q_0: q_wa,
        T_0: T_wa,
        T_inf: T_infa,
        x_0: x_wa,
    }
)
T_a_expr = T_a_int_expr.subs(
    {
        q_wa: q_wa_expr_w,
        T_wa: T_wa_expr_w,
    }
)

T_wa_expr_a = T_a_expr.subs(x, x_wa)
q_wa_expr_a = (
    T_a_expr.diff(x).subs(  # type: ignore  # sympy
        x,
        x_wa,
    )
    * k
)

disp_free("Solution in the air domain", Eq(T_a(x), T_a_int_expr))
disp_free("Solution in the air domain, with substitutions", Eq(T_a(x), T_a_expr))

## Piecewise temperature distribution


In [ ]:
T_expr = Piecewise(
    (T_w_expr, x < x_wa),
    (T_a_expr, True),
)

disp_free("Temperature distribution in the rod", Eq(T(x), T_expr))

## Make the model function and serialize


In [ ]:
class Models:
    cm2_p_m2 = 100**2  # ((cm/m)^2) Conversion factor

    def __init__(self):
        """Model of the temperature distribution in a rod.

        Consists of the lambdified model function and a method for generating a wrapped
        model function that has more flexible input requirements.
        """

        expr = T_expr.evalf(
            subs=PARAMS.fit.model_inputs
            | {q_s: q_s * self.cm2_p_m2}  # (W/m^2) = (W/cm^2 * cm^2/m^2)
        )
        disp_free("Temperature distribution after float evaluation", Eq(T(x), expr))

        overrides = {
            ufun.name: np.vectorize(ufun)
            for ufun in (
                umath.exp,  # type: ignore  # uncertainties
                umath.sqrt,  # type: ignore  # uncertainties
            )
        }

        self.basic = lambdify(args=params, expr=expr, modules=np)
        self.for_ufloat = lambdify(args=params, expr=expr, modules=[overrides, np])


models = Models()

with warnings.catch_warnings():
    warnings.simplefilter("ignore", dill.PicklingWarning)
    PARAMS.paths.file_model.write_bytes(dill.dumps(models))